# 🌟데이터 EDA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import cv2
import seaborn as sns
import numpy as np
import os
import warnings
from glob import glob
import json
warnings.filterwarnings('ignore')

path = '/aiffel/aiffel/aiffelthon/데이터EDA'
# csv가져오기
meta = pd.read_csv(path+'/meta.csv')
anno = pd.read_csv(path+'/anno.csv')

In [ ]:
meta.head()

In [ ]:
anno.head()

## 💡meta 데이터 분석

In [ ]:
# 이미지 size 확인
meta['size'].value_counts()

In [ ]:
# 날씨 확인
meta['weather'].value_counts()

In [ ]:
# 도로 특징 확인
meta['road_feature'].value_counts()

In [ ]:
# 위치 특징 확인
meta['location_feature'].value_counts().sort_index()

In [ ]:
# 운전 상황 확인
meta['driving_scenario'].value_counts()

In [ ]:
# 도로 타입 확인
meta['road_type'].value_counts()

In [ ]:
# 운전 속도 확인
meta['ego_long_vel_level'].value_counts()

In [ ]:
# 조도(빛) 상태 확인
meta['illumination_status'].value_counts()

In [ ]:
# 도로 상태 확인
meta['road_status'].value_counts()

In [ ]:
# 객체 수 확인
meta['crowd_level'].value_counts()

In [ ]:
# 센서 상태 확인
meta['sensor_status'].value_counts()

## 📈meta 데이터 시각화

In [ ]:
plot_list = ['weather','sensor_status','crowd_level','illumination_status']
plt.figure(figsize=(30, 20))
# plt.subplots(constrained_layout=True)
palette = sns.color_palette('coolwarm', 10)
for i in range(len(plot_list)):
    plt.subplot(2,2,i+1)
    ax = sns.countplot(x = plot_list[i], data = meta, palette = palette)
#     plt.title(plot_list[i],fontsize =15)
    plt.xlabel(plot_list[i],fontsize = 18)
    plt.ylabel('Rates',fontsize = 18)
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x() + p.get_width()/2., height+5, height, ha = 'center', size = 17)

    
# 원-핫 
df = pd.get_dummies(meta[['weather','sensor_status','crowd_level','illumination_status']])
df = df.corr()
# 그림 사이즈 지정
fig, ax = plt.subplots( figsize=(10,10) )

# 삼각형 마스크를 만든다(위 쪽 삼각형에 True, 아래 삼각형에 False)
mask = np.zeros_like(df, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# 히트맵을 그린다
sns.heatmap(df, 
            cmap = 'RdYlBu_r', 
            annot = True,   # 실제 값을 표시한다
            mask=mask,      # 표시하지 않을 마스크 부분을 지정한다
            linewidths=.5,  # 경계면 실선으로 구분하기
            cbar_kws={"shrink": .5},# 컬러바 크기 절반으로 줄이기
            vmin = -1,vmax = 1   # 컬러바 범위 -1 ~ 1
           )
plt.show()

## 📝 meta 데이터 분석 결과
- 1. 날씨 
    - 날씨의 경우 날씨에 따른 여러가지 상태를 고려해야 할 것 같다.
    - 맑은 날은 역광이, 흐린날에는 물맺힘 현상등을 고려해야 할거 같고 강건화를 위해서는 최대한 포함시키는게 좋을것 같다.
- 2. 객체 수
    - 자율주행의 관점에서 고려해 보았을때 객체가 밀집한 경우는 보통 비슷한 객체가 모여있을 확률이 높고 그중 하나만 detect된다면 굳이 모든 객체를 분류 할 필요는 없을 것으로 보인다.
    - 학습의 관점에서 보았을 때는 분류학습을 위해 최대한 객체가 많이 포함되는것이 좋아보인다.
    - 결론적으로 객체가 도로에 많아서 학습에 악영향이 있다고 보이진 않고 학습을 위해서 최대한 객체 를 많이 포함하는 방향으로 가는게 좋아 보인다.
- 3. 센서상태, 역광
    - 해당문제는 날씨와 연관이 있다고 생각했는데 상관관계를 보니 그렇지는 않은것으로 나타났다.
    - 센서에 물이 맷혀도, 역광이 심해도 우리는 객체 인식을 잘 해야하기 때문에 최대한 모든데이터를 골고루 포함하도록 해야할것 같다. 

## 💡annotation 분석

In [ ]:
# label 확인
anno['label'].value_counts()

In [ ]:
# attribute 확인
anno['attribute'].value_counts()

In [ ]:
# occlusion(객체가 가려진 상태)
anno['occlusion'].value_counts()

In [ ]:
# truncation(객체가 잘린 상태) 확인
anno['truncation'].value_counts()

In [ ]:
# scenario(객체의 거동 시나리오) 확인
anno['scenario'].value_counts()

In [ ]:
# isfake(객체의 진위 여부) 확인
anno['isfake'].value_counts()

In [ ]:
# ismask(마스크 착용 여부) 확인
anno['ismask'].value_counts()

## 📈annotations 시각화

In [ ]:
plot_list = ['label','attribute','occlusion','truncation','isfake']
plt.figure(figsize=(30, 20))
# plt.subplots(constrained_layout=True)
for i in range(len(plot_list)):
    plt.subplot(3,2,i+1)
    ax = sns.countplot(x = plot_list[i], data = anno, palette = palette)
    plt.title(plot_list[i],fontsize =20)
    plt.xlabel(plot_list[i],fontsize = 20)
    plt.ylabel('Rates',fontsize = 20)
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x() + p.get_width()/2., height+5, height, ha = 'center', size = 16)

## 📝annotations 데이터 분석 결과
- 라벨, attribute, 가려진상태, 잘린 상태, 가짜이미지 들에 대해 분석해 보았다.
- 1. 라벨 
    - 클래스를 라벨로 할지 attr로 할지 고민 해 보아야 할것 같고 객체의 분포가 일정하지 않아 객체수가 상대적으로 적은것들은 학습이 잘 되지 않을 것으로 보이므로 최대한 적은객체를 다포함시키거나 배제할 수 있으면 배재하는 방향으로 가야할 것 같다.
- 2. occlusion, truncation
    - 강건화를 위해서 잘리거나 가려진 이미지를 많이 포함하는 방향으로 데이터를 구성해야 할것 같다.
- 3. isfake
    - 가짜 이미지 시각화해서 분석후 포함할지 결정을 해야 할것 같다.

# 📊이미지 시각화  (smear, pm, is fake)

## 🎈weather:['sunny','cloudy']

In [ ]:
# sunny
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    if (info['weather'] ==  'sunny')&(info['filename']=='1655166871340_FR-View-CMR-Wide.png'):
        plt.figure(figsize=(12,8))
        img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
        plt.title(f"{info['weather']}")
#         print(info['filename'])
        plt.imshow(img)
        plt.show()
        break

In [ ]:
# cloudy
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))

for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    if (info['weather'] ==  'cloudy')&(info['filename'] == '1654740559733_FR-View-CMR-Wide.png'):
        plt.figure(figsize=(12,8))
        img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
        plt.title(f"{info['weather']}")
        plt.imshow(img)
        plt.show()
        break

## 🎈sensor_status:['normal', 'reflection_from_dashboard', 'waterdrop', 'smear']

In [ ]:
# normal
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))

c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    if (info['sensor_status'] ==  'normal')&(info['filename'] == '1655165423198_FR-View-CMR-Wide.png'):
        plt.figure(figsize=(12,8))
        img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
        plt.title("sensor_status:normal")
        plt.imshow(img)
        plt.show()
        break

In [ ]:
# reflection_from_dashboard
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))

c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    if (info['sensor_status'] ==  'reflection_from_dashboard')&(info['filename']=='1654739281199_FR-View-CMR-Wide.png'):
        plt.figure(figsize=(12,8))
        img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
#         print(info['filename'])
        plt.title("sensor_status:reflection_from_dashboard")
        plt.imshow(img)
        plt.show()
        break

In [ ]:
# waterdrop
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))

c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    if (info['sensor_status'] ==  'waterdrop')&(info['filename']=='1656978553854_FR-View-CMR-Wide.png'):
        plt.figure(figsize=(12,8))
        img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
#         print(info['filename'])
        plt.title("sensor_status:waterdrop")
        plt.imshow(img)
        plt.show()
        break

In [ ]:
# smear
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))

c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    if (info['sensor_status'] ==  'smear')&(info['filename']=='1659682286392_FR-View-CMR-Wide.png'):
        plt.figure(figsize=(12,8))
        img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
#         print(info['filename'])
        plt.title("sensor_status:smear")
        plt.imshow(img)
        plt.show()
        break

## 🎈illumination_status:['backlight_sun']

In [ ]:
# backlight_sun
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))

c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    if (info['illumination_status'] ==  'backlight_sun')&(info['filename']=='1657874329008_FR-View-CMR-Wide.png'):
        plt.figure(figsize=(12,8))
        img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
        plt.title("illumination_status:backlight_sun")
        plt.imshow(img)
        plt.show()
        break

## 🎈crowd_level :['high', 'middle', 'low']

In [ ]:
# high 
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))

c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    if( info['crowd_level'] ==  'high')&(info['filename']=='1654741032452_FR-View-CMR-Wide.png'):
        plt.figure(figsize=(12,8))
        img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
#         print(info['filename'])
        plt.title("crowd_level :high")
        plt.imshow(img)
        plt.show()
        break

In [ ]:
# middle 
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))

c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    if (info['crowd_level'] ==  'middle')&(info['filename']=='1654740815439_FR-View-CMR-Wide.png'):
        plt.figure(figsize=(12,8))
        img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
#         print(info['filename'])
        plt.title("crowd_level :middle")
        plt.imshow(img)
        plt.show()
        break

In [ ]:
# low 
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))

c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    if (info['crowd_level'] ==  'low')&(info['filename']=='1656049121722_FR-View-CMR-Wide.png'):
        plt.figure(figsize=(12,8))
        img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
#         print(info['filename'])
        plt.title("crowd_level :low")
        plt.imshow(img)
        plt.show()
        break

### 🎈attribute:['pm','face','unknown','trailer','specialclothes','movable_object','wheelchair_stroller','animal']

In [ ]:
# pm
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['attribute'] == 'pm')&(info['filename']=='1656046860751_RR-Right-View-CMR-Narrow.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
#             print(info['filename'])
            plt.title('attrubute:pm')
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c==1:
        break

In [ ]:
# face
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['attribute'] == 'face')&(info['filename']=='1656044116008_RR-View-CMR-Wide.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
#             print(info['filename'])
            plt.title('attrubute:face')
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c==1:
        break
            

In [ ]:
# unknown
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['attribute'] == 'unknown')&(info['filename']=='1654759830752_FR-View-CMR-Wide.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
#             print(info['filename'])
            plt.title('attrubute:unknown')
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c==1:
        break

In [ ]:
# trailer
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['attribute'] == 'trailer')&(info['filename']=='1657073055665_FR-View-CMR-Wide.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
#             print(info['filename'])
            plt.title('attrubute:trailer')
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c>=1:
        break
            

In [ ]:
# specialclothes
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['attribute'] == 'specialclothes')&(info['filename']=='1656044112975_RR-Left-View-CMR-Narrow.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
#             print(info['filename'])
            plt.title('attrubute:specialclothes')
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c>=1:
        break
            

In [ ]:
# movable_object
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['attribute'] == 'movable_object')&(info['filename']=='1656050309533_RR-Right-View-CMR-Narrow.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
#             print(info['filename'])
            plt.title('attrubute:movable_object')
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c>=1:
        break
            

In [ ]:
# wheelchair_stroller
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['attribute'] == 'wheelchair_stroller')&(info['filename']=='1654752917404_FR-View-CMR-Wide.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
#             print(info['filename'])
            plt.title('attrubute:wheelchair_stroller')
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c>=1:
        break

In [ ]:
# animal
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['attribute'] == 'animal')&(info['filename']=='1659687907847_FR-View-CMR-Wide.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
#             print(info['filename'])
            plt.title('attrubute:animal')
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c>=1:
        break

## 🎈occlusion:[1,2,3,4] 

In [ ]:
# 1
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['occlusion'] == 1)&(info['filename']=='1654739510387_FR-View-CMR-Wide.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
            plt.title('occlusion:1')
#             print(info['filename'])
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c>=1:
        break

In [ ]:
# 2
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['occlusion'] == 2)&(info['filename']=='1654751972944_FR-View-CMR-Wide.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
            plt.title('occlusion:2')
#             print(info['filename'])
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c>=1:
        break

In [ ]:
# 3
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['occlusion'] == 3)&(info['filename']=='1654740559733_FR-View-CMR-Wide.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
            plt.title('occlusion:3')
#             print(info['filename'])
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c>=1:
        break

In [ ]:
# 4
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['occlusion'] == 4)&(info['filename']=='1654741503842_FR-View-CMR-Wide.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
            plt.title('occlusion:4')
#             print(info['filename'])
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c>=1:
        break

## 🎈truncation:[1,2,3,4]

In [ ]:
# 1
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['truncation'] == 1)&(info['filename']=='1654752725510_FR-View-CMR-Wide.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
            plt.title('truncation:1')
#             print(info['filename'])
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c>=1:
        break

In [ ]:
# 2
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['truncation'] == 2)&(info['filename']=='1655165541867_FR-View-CMR-Wide.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
            plt.title('truncation:2')
#             print(info['filename'])
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c>=1:
        break

In [ ]:
# 3
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['truncation'] == 3)&(info['filename']=='1654740559733_FR-View-CMR-Wide.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
            plt.title('truncation:3')
#             print(info['filename'])
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c>=1:
        break

In [ ]:
# 4
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if (k['truncation'] == 4)&(info['filename']=='1654741032452_FR-View-CMR-Wide.png'):
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
            plt.title('truncation:4')
#             print(info['filename'])
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c>=1:
        break

## 🎈isfake:[1]

In [ ]:
# is_fake
path = '/aiffel/aiffel/aiffelthon/데이터EDA/random_1000/'
png_list = sorted(glob(path+'*.png'))
json_list = sorted(glob(path+'*.json'))
c=0
for i, j in zip(png_list, json_list):
    with open(j,'r')as f:
        info = json.load(f)
    for k in info['annotations']:
        if k['isfake'] == 1:
            plt.figure(figsize=(12,8))
            img = cv2.cvtColor( cv2.imread(i), cv2.COLOR_BGR2RGB)
            img = cv2.rectangle(img, k['points'][0], k['points'][2], (255,0,255), 2)
            plt.title('isfake:1')
            plt.imshow(img)
            plt.show()
            c+=1
            break
    if c>=1:
        break

## 📝시각화 분석 결과
- <h3>weather(날씨)</h3>
    - sunny:맑음, cloudy:흐림
- <h3>sensor_status(센서상태)</h3>
    - normal:일반, reflection_from_dashboard:대시보드 반사, waterdrop:물맺힘, smear빛 번짐
- <h3>illumination_status(조도 상태)</h3>
    - backlight_sun:역광
- <h3>crowd_level(객체수)</h3>
    - high: 10 이상, middle : 5~10 사이, low: 5이하
- <h3>attribute (특성)</h3>
    - pm:(차량을 제외한 주차되어 있는 물체(킥보드, 자전거, 오토바이 등)), face:얼굴, unknown:미분류 객체, trailer:트레일러, specialclothes:안전요원, movable_object:수레,리어카 등,  wheelchair_stroller:휠체어,유모차등, animal:동물
- <h3>occlusion(가려짐)</h3>
    - 가려진 객체
- <h3>truncation(잘림)</h3>
    - 잘린 객체
- <h3>isfake(가짜)</h3>
    - 가짜 객체 , 현수막, 광고판에 나오는 객체등